# 입문자를 위한, 파이썬/R 데이터 분석  

]

## Today's mission

- ETF(상장지수펀드) 목록 수집
- <p style="font-weight:bold">ETF 데이터 전처리</p>
- ETF 분석과 시각화

]

## ETF 데이터 전처리

* [ETF : 네이버 금융](https://finance.naver.com/sise/etf.nhn) 에서 수집한 데이터를 전처리 합니다.
* keyword
    * rename column
    * .map(lambda x : x)
    * text data
        * str.contains()
        * str.replace()
        * str.split("구분자", expand=True)
        * str[:]

### 데이터 로드와 요약

In [1]:
# 데이터 분석을 위한 pandas, 수치계산을 위한 numpy라이브러리를 로드합니다.
import pandas as pd
import numpy as np

In [2]:
# 이전 실습에서 저장한 파일명을 file_name에 적어주세요.
import datetime

today = datetime.datetime.today()
today = today.strftime("%Y-%m-%d")
today

file_name = f"data/etf_{today}_raw.csv"
file_name

'data/etf_2024-08-23_raw.csv'

In [3]:
# 이전 수업에서 수집해서 저장해둔 csv 파일을 읽어옵니다.
# itemcode 숫자 앞의 0 이 지워진다면 dtype={"itemcode": np.object} 로 타입을 지정해 주면 문자형태로 읽어옵니다.
df = pd.read_csv(file_name, dtype={"itemcode":object})
df.shape

(881, 12)

In [4]:
# 인덱스 번호 상단 5개의 데이터를 가져옵니다.
# 데이터를 잘 읽어왔는지 확인합니다.
df.head()

,itemcode,etfTabCode,itemname,nowVal,risefall,changeVal,changeRate,nav,threeMonthEarnRate,quant,amonut,marketSum
0,459580,6,KODEX CD금리액티브(합성),1046085,2,230,0.02,1046089.0,0.8972,195638,204654,93402
1,069500,1,KODEX 200,36820,5,-115,-0.31,36885.0,-0.4931,1101467,40510,61140
2,357870,6,TIGER CD금리투자KIS(합성),54825,3,0,0.00,54828.0,0.9110,25319,1388,61007
3,423160,6,KODEX KOFR금리액티브(합성),107820,2,30,0.03,107779.0,0.8939,16807,1812,45037
4,360750,4,TIGER 미국S&P500,18650,5,-35,-0.19,18599.0,3.6730,4820460,89909,43184


In [5]:
# 인덱스 하단의 5개 데이터를 가져옵니다.
# 데이터를 잘 읽어왔는지 확인합니다.
df.tail()

,itemcode,etfTabCode,itemname,nowVal,risefall,changeVal,changeRate,nav,threeMonthEarnRate,quant,amonut,marketSum
876,267450,6,RISE 미국장기국채선물인버스(H),10160,3,0,0.00,10216.0,-5.6639,0,0,20
877,412560,2,TIGER BBIG레버리지,2190,5,-10,-0.45,2199.0,0.9174,1411,3,18
878,333980,2,PLUS KS퀄리티가중TR,15230,2,20,0.13,15266.0,0.3960,4,0,15
879,436180,2,PLUS K-유니콘투자기업액티브,9975,2,25,0.25,9987.0,-3.5853,1,0,10
880,402520,1,FOCUS 혁신기업액티브,8795,3,0,0.00,8840.0,-1.4014,0,0,9


In [6]:
# info를 통해서 각 column들의 데이터타입과 결측치, 메모리 사용량 등을 볼 수 있습니다.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881 entries, 0 to 880
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   itemcode            881 non-null    object 
 1   etfTabCode          881 non-null    int64  
 2   itemname            881 non-null    object 
 3   nowVal              881 non-null    int64  
 4   risefall            881 non-null    int64  
 5   changeVal           881 non-null    int64  
 6   changeRate          881 non-null    float64
 7   nav                 881 non-null    float64
 8   threeMonthEarnRate  839 non-null    float64
 9   quant               881 non-null    int64  
 10  amonut              881 non-null    int64  
 11  marketSum           881 non-null    int64  
dtypes: float64(3), int64(7), object(2)
memory usage: 82.7+ KB


* 웹사이트의 ETF 정보와 비교하며 어떤 컬럼이 어떤 값인지 확인합니다. 
* https://finance.naver.com/sise/etf.nhn 

In [7]:
# https://finance.naver.com/sise/etf.nhn 에서 값을 비교해보면 quant는 거래량임을 알 수 있습니다.
# 거래량(quant) 기준으로 내림차순으로 정렬하여 상위 10개의 데이터를 봅니다.
df.sort_values("quant", ascending=False).head(10)

,itemcode,etfTabCode,itemname,nowVal,risefall,changeVal,changeRate,nav,threeMonthEarnRate,quant,amonut,marketSum
22,252670,3,KODEX 200선물인버스2X,1989,2,14,0.71,1991.0,-1.9852,70732449,141032,13054
99,251340,3,KODEX 코스닥150선물인버스,3695,2,15,0.41,3709.0,1.7980,12432953,46131,3248
56,114800,3,KODEX 인버스,4160,2,15,0.36,4161.0,0.2418,7432640,30963,5874
21,233740,3,KODEX 코스닥150레버리지,9125,5,-65,-0.71,9187.0,-12.1415,5627739,50934,13186
4,360750,4,TIGER 미국S&P500,18650,5,-35,-0.19,18599.0,3.6730,4820460,89909,43184
11,122630,3,KODEX 레버리지,18745,5,-120,-0.64,18902.0,-4.1900,4098125,76627,21332
137,462330,2,KODEX 2차전지산업레버리지,2350,5,-65,-2.69,2363.0,-28.9706,1441541,3391,2200
283,252710,3,TIGER 200선물인버스2X,2110,2,20,0.96,2106.0,-2.3365,1331676,2812,724
1,069500,1,KODEX 200,36820,5,-115,-0.31,36885.0,-0.4931,1101467,40510,61140
15,379800,4,KODEX 미국S&P500TR,17040,5,-35,-0.20,17000.0,3.6733,1098057,18717,16733


In [8]:
# etfTabCode column의 데이터 구성을 살펴봅니다.
# 추후 알게되겠지만 etfTabCode는 해당 사이트에서 
# 전체(0), 국내 시장지수(1), 국내 업종/테마(2), 국내 파생(3) ~
# 해외 주식(4), 원자재(5), 채권(6), 기타(7)로 자료가 구분되어 있습니다.
df["etfTabCode"].value_counts()#.sort_index()

etfTabCode
4    269
2    245
6    158
7     76
1     75
3     40
5     18
Name: count, dtype: int64

In [12]:
# pandas 의 boolean Indexing을 사용해서
# 국내 시장지수(etfTabCode == 1)의 데이터들만 확인하여 봅니다.
# 조건문의 결과가 Index와  True, False 로 나오기 때문에 boolean Indexing 이라 부릅니다.
df[df["etfTabCode"] == 1]
# df[df["etfTabCode"] == 4].sort_values('quant', ascending=False)

,itemcode,etfTabCode,itemname,nowVal,risefall,changeVal,changeRate,nav,threeMonthEarnRate,quant,amonut,marketSum
1,069500,1,KODEX 200,36820,5,-115,-0.31,36885.0,-0.4931,1101467,40510,61140
10,278530,1,KODEX 200TR,12855,5,-40,-0.31,12880.0,-0.5400,8870,113,22657
14,102110,1,TIGER 200,36880,5,-110,-0.30,36949.0,-0.5325,348733,12843,19712
25,148020,1,RISE 200,37075,5,-135,-0.36,37139.0,-0.4362,112656,4171,12494
27,310970,1,TIGER MSCI Korea TR,15755,5,-45,-0.28,15788.0,0.1902,12,0,12179
...,...,...,...,...,...,...,...,...,...,...,...,...
816,391670,1,HK 베스트일레븐액티브,9690,3,0,0.00,9743.0,0.3625,0,0,48
818,433250,1,UNICORN R&D 액티브,11990,5,-50,-0.42,11994.0,0.0831,13,0,48
824,391680,1,HK 하이볼액티브,8165,3,0,0.00,8179.0,-2.8555,0,0,45
837,301400,1,PLUS 코스닥150,13030,5,-30,-0.23,13081.0,-4.4973,19,0,39


### 데이터 전처리
### etfTabName 만들기

In [13]:
print("""전체
국내 시장지수
국내 업종/테마
국내 파생
해외 주식
원자재
채권
기타""".split("\n"))

['전체', '국내 시장지수', '국내 업종/테마', '국내 파생', '해외 주식', '원자재', '채권', '기타']


In [14]:
# etf name을 구분하기 위한 list를 만듭니다.
# """ 를 이용해 \n을 포함하는 string을 만들고 split으로 나누어서 list를 간편히 만들수 있습니다.
etfcode = """전체
국내 시장지수
국내 업종/테마
국내 파생
해외 주식
원자재
채권
기타"""
etfcode

'전체\n국내 시장지수\n국내 업종/테마\n국내 파생\n해외 주식\n원자재\n채권\n기타'

In [15]:
# split으로 나누어 list형 데이터를 만듭니다.
etf_tab_name = etfcode.split("\n")
etf_tab_name

['전체', '국내 시장지수', '국내 업종/테마', '국내 파생', '해외 주식', '원자재', '채권', '기타']

In [16]:
def find_etf_tab_name(no):
    return etf_tab_name[no]

In [17]:
find_etf_tab_name(2)

'국내 업종/테마'

In [18]:
df["etfTabCode"].map(lambda x : etf_tab_name[x])

0            채권
1       국내 시장지수
2            채권
3            채권
4         해외 주식
         ...   
876          채권
877    국내 업종/테마
878    국내 업종/테마
879    국내 업종/테마
880     국내 시장지수
Name: etfTabCode, Length: 881, dtype: object

In [19]:
# etfTabName 이름이 직관적이지 않기 때문에 한글로 변경해 줍니다.
# map과 lambda 함수를 이용하여 etfTabCode column들의 각 cell의 내용에 따라
# etf_tab_name list의 원소값에 따라 이름을 만들어 주고 etfTabName 이라는 새로운 컬럼을 생성합니다.
# 즉 etfTabCode 숫자 -> list의 원소 인덱스로 한글 이름을 매핑해 줍니다. -> etfTabName에 한글로 저장
df["etfTabName"] = df["etfTabCode"].map(lambda x : etf_tab_name[x])

In [20]:
# etfTabName column이 제대로 만들어졌는지 확인합니다.
df.loc[df["etfTabCode"] == 1, ["itemcode", "etfTabCode", "itemname", "etfTabName"]]#.head()

,itemcode,etfTabCode,itemname,etfTabName
1,069500,1,KODEX 200,국내 시장지수
10,278530,1,KODEX 200TR,국내 시장지수
14,102110,1,TIGER 200,국내 시장지수
25,148020,1,RISE 200,국내 시장지수
27,310970,1,TIGER MSCI Korea TR,국내 시장지수
...,...,...,...,...
816,391670,1,HK 베스트일레븐액티브,국내 시장지수
818,433250,1,UNICORN R&D 액티브,국내 시장지수
824,391680,1,HK 하이볼액티브,국내 시장지수
837,301400,1,PLUS 코스닥150,국내 시장지수


* map : 
    * Series에서만 사용가능
    * [Essential basic functionality — pandas documentation](https://pandas.pydata.org/docs/user_guide/basics.html?#applying-elementwise-functions)

* apply :
    * Series와 DataFrame 둘 다 사용가능
    * [Essential basic functionality — pandas documentation](https://pandas.pydata.org/docs/user_guide/basics.html?#row-or-column-wise-function-application)
    
참고 : http://www.leejungmin.org/post/2018/04/21/pandas_apply_and_map/

### 컬럼명 변경하기

In [21]:
"""종목명
현재가
전일비
등락률
NAV
3개월수익률
거래량
거래대금(백만)
시가총액(억)"""

'종목명\n현재가\n전일비\n등락률\nNAV\n3개월수익률\n거래량\n거래대금(백만)\n시가총액(억)'

In [22]:
df.columns

Index(['itemcode', 'etfTabCode', 'itemname', 'nowVal', 'risefall', 'changeVal',
       'changeRate', 'nav', 'threeMonthEarnRate', 'quant', 'amonut',
       'marketSum', 'etfTabName'],
      dtype='object')

In [23]:
# DataFrame df의 column 이름을 list로 만들어서 cols 라는 변수에 담습니다.
cols = df.columns.tolist()
cols

['itemcode',
 'etfTabCode',
 'itemname',
 'nowVal',
 'risefall',
 'changeVal',
 'changeRate',
 'nav',
 'threeMonthEarnRate',
 'quant',
 'amonut',
 'marketSum',
 'etfTabName']

In [24]:
# 영어로 되어있는 column 이름을 한글로 바꾸기 위한 list를 만듭니다.
# 빠진 column이 있는지 리스트와 길이를 확인합니다.
col_name = """종목코드
탭코드
종목명
현재가
등락구분
전일비
등락률
순자산가치(NAV)
3개월수익률
거래량
거래대금(백만)
시가총액(억)
유형"""
col_name = col_name.split("\n")
col_name

['종목코드',
 '탭코드',
 '종목명',
 '현재가',
 '등락구분',
 '전일비',
 '등락률',
 '순자산가치(NAV)',
 '3개월수익률',
 '거래량',
 '거래대금(백만)',
 '시가총액(억)',
 '유형']

- 참고 : [zip() 함수](https://ooyoung.tistory.com/60)

In [25]:
zip(cols, col_name)

In [26]:
# 컬럼 영문명과 한글명을 딕셔너리 형태로 만들어 비교해 볼 수도 있습니다.
# 하지만 좀 더 간단하게 여기에서는 리스트 값을 그대로 컬럼에 넣어주는 방법을 사용할거에요.
# 딕셔너리로 키-값 쌍을 만들어 컬럼의 키와 값이 잘 매치되었는지 확인해 봅니다.
dict(zip(cols, col_name))

{'itemcode': '종목코드',
 'etfTabCode': '탭코드',
 'itemname': '종목명',
 'nowVal': '현재가',
 'risefall': '등락구분',
 'changeVal': '전일비',
 'changeRate': '등락률',
 'nav': '순자산가치(NAV)',
 'threeMonthEarnRate': '3개월수익률',
 'quant': '거래량',
 'amonut': '거래대금(백만)',
 'marketSum': '시가총액(억)',
 'etfTabName': '유형'}

In [27]:
# 기존의 데이터프레임 컬럼명에 위에서 만든 컬럼명을 할당연산자로 대입해 주면 컬럼명이 변경됩니다.
# 컬럼 변경 후 한글로 컬럼명이 잘 변경되었는지 확인해 봅니다.
df.columns = col_name
df.head()

,종목코드,탭코드,종목명,현재가,등락구분,전일비,등락률,순자산가치(NAV),3개월수익률,거래량,거래대금(백만),시가총액(억),유형
0,459580,6,KODEX CD금리액티브(합성),1046085,2,230,0.02,1046089.0,0.8972,195638,204654,93402,채권
1,069500,1,KODEX 200,36820,5,-115,-0.31,36885.0,-0.4931,1101467,40510,61140,국내 시장지수
2,357870,6,TIGER CD금리투자KIS(합성),54825,3,0,0.00,54828.0,0.9110,25319,1388,61007,채권
3,423160,6,KODEX KOFR금리액티브(합성),107820,2,30,0.03,107779.0,0.8939,16807,1812,45037,채권
4,360750,4,TIGER 미국S&P500,18650,5,-35,-0.19,18599.0,3.6730,4820460,89909,43184,해외 주식


### 파생변수 만들기
* 브랜드, 인버스, 레버리지, 환헤지H 변수 만들기
* [Working with text data — pandas documentation](https://pandas.pydata.org/docs/user_guide/text.html)

In [28]:
str.split?

Signature: str.split(self, /, sep=None, maxsplit=-1)
Docstring:
Return a list of the words in the string, using sep as the delimiter string.

sep
  The delimiter according which to split the string.
  None (the default value) means split according to any whitespace,
  and discard empty strings from the result.
maxsplit
  Maximum number of splits to do.
  -1 (the default value) means no limit.
Type:      method_descriptor

In [29]:
"KODEX 200".split()#[0]

['KODEX', '200']

In [30]:
"KODEX 200".split()[1]

'200'

In [31]:
df["종목명"].str.split(" ", expand=True)[0]

0      KODEX
1      KODEX
2      TIGER
3      KODEX
4      TIGER
       ...  
876     RISE
877    TIGER
878     PLUS
879     PLUS
880    FOCUS
Name: 0, Length: 881, dtype: object

In [32]:
df["종목명"].str.split(expand=True)

,0,1,2,3
0,KODEX,CD금리액티브(합성),None,None
1,KODEX,200,None,None
2,TIGER,CD금리투자KIS(합성),None,None
3,KODEX,KOFR금리액티브(합성),None,None
4,TIGER,미국S&P500,None,None
...,...,...,...,...
876,RISE,미국장기국채선물인버스(H),None,None
877,TIGER,BBIG레버리지,None,None
878,PLUS,KS퀄리티가중TR,None,None
879,PLUS,K-유니콘투자기업액티브,None,None


In [33]:
# 종목명 column의 데이터를 space(" ")를 이용해 나누고 제일 앞부분[0]을 새로운 column을 만들어 저장하고 확인합니다.
# expand = True 옵션을 사용하면 문자열을 나눈 값을 인덱스 순서대로 가져와서 사용할 수 있습니다.
# df["브랜드"]
df["브랜드"] = df["종목명"].str.split(" ", expand=True)[0]
df[["브랜드", "종목명"]].head()

,브랜드,종목명
0,KODEX,KODEX CD금리액티브(합성)
1,KODEX,KODEX 200
2,TIGER,TIGER CD금리투자KIS(합성)
3,KODEX,KODEX KOFR금리액티브(합성)
4,TIGER,TIGER 미국S&P500


* 종목명 column의 내용중이 '인버스'라는 단어가 있으면 새로운 column '인버스'에 True 값이 들어가게 됩니다.
* contains는 bool type을 반환하기 때문에 새로운 column에는 True나 False가 들어가게 됩니다.
* 마찬가지로 '레버리지'와 '환헤지H'에 대해서도 동일하게 만들어 줍니다. 
* H라는 단어가 종목명에 들어갈 수도 있기 때문에 H뒤에 닫는 소괄호")"까지 확인합니다.
* 소괄호는 정규표현식에서 의미를 가지는 문자이기 때문에 문자 그대로 읽어오기 위해서는 "\"로 전처리가 필요합니다.

In [34]:
df["종목명"].str.contains("인버스")

0      False
1      False
2      False
3      False
4      False
       ...  
876     True
877    False
878    False
879    False
880    False
Name: 종목명, Length: 881, dtype: bool

In [35]:
# 인버스ETF는 지수가 하락하면 오히려 수익률이 오르도록 설계된 상품입니다.
df["인버스"] = df["종목명"].str.contains("인버스")
df["인버스"].value_counts(normalize=True) * 100

인버스
False    95.005675
True      4.994325
Name: proportion, dtype: float64

레버리지(leverage)는 타인의 자본을 지렛대처럼 이용하여 자기 자본의 이익률을 높이는 것이다.

* 출처 : [레버리지](https://ko.wikipedia.org/wiki/%EC%83%81%EC%9E%A5%EC%A7%80%EC%88%98_%ED%8E%80%EB%93%9C#%EB%A0%88%EB%B2%84%EB%A6%AC%EC%A7%80_ETF)

In [36]:
df["레버리지"] = df["종목명"].str.contains("레버리지")
df["레버리지"].value_counts(normalize=True) * 100

레버리지
False    94.551646
True      5.448354
Name: proportion, dtype: float64

헤지(hedge)란 환율, 금리 또는 다른 자산에 대한 투자등을 통해 보유하고 있는 위험자산의 가격변동을 제거하는 것을 말한다. 즉, 확정되지 않은 자산을 확정된 자산으로 편입하는 과정이라 할 수 있으며, 주로 선물 옵션과 같은 파생상품을 이용한다. 이를 통해 체계적 위험을 제거할 수 있다.

부(wealth)를 결정하는 변수값의 변화와 관계없이 항상 일정한 부를 유지하게 하는 헤지를 완전헤지라고 하고, 그렇지 못한 것을 불완전헤지라고 한다.
* 출처 : [환헤지](https://m.post.naver.com/viewer/postView.nhn?volumeNo=30732948&memberNo=32787516)

In [37]:
# (H)의 의미는 환율변동 위험을 막기 위해 Hedge를 한다는 뜻이다.
df["환헤지H"] = df["종목명"].str.endswith("H)")
df["환헤지H"].value_counts(normalize=True) * 100

환헤지H
False    89.330306
True     10.669694
Name: proportion, dtype: float64

In [38]:
df.loc[df["환헤지H"] == True, ["종목명", "환헤지H"]].head()

,종목명,환헤지H
18,ACE 미국30년국채액티브(H),True
43,TIGER 미국30년국채프리미엄액티브(H),True
60,KODEX 미국빅테크10(H),True
62,TIGER 미국30년국채스트립액티브(합성 H),True
97,RISE 미국30년국채엔화노출(합성 H),True


### 전처리가 잘 되었는지 확인하기

In [39]:
# 등락구분에 있는 값의 빈도수를 세어봅니다.
# 5 : -, 2 : +, 3 : 0
df["등락구분"].value_counts()

등락구분
5    490
2    302
3     89
Name: count, dtype: int64

In [40]:
# 전체적으로 데이터 전처리가 끝났다면 info를 통해 데이터 요약정보 보기
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881 entries, 0 to 880
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   종목코드        881 non-null    object 
 1   탭코드         881 non-null    int64  
 2   종목명         881 non-null    object 
 3   현재가         881 non-null    int64  
 4   등락구분        881 non-null    int64  
 5   전일비         881 non-null    int64  
 6   등락률         881 non-null    float64
 7   순자산가치(NAV)  881 non-null    float64
 8   3개월수익률      839 non-null    float64
 9   거래량         881 non-null    int64  
 10  거래대금(백만)    881 non-null    int64  
 11  시가총액(억)     881 non-null    int64  
 12  유형          881 non-null    object 
 13  브랜드         881 non-null    object 
 14  인버스         881 non-null    bool   
 15  레버리지        881 non-null    bool   
 16  환헤지H        881 non-null    bool   
dtypes: bool(3), float64(3), int64(7), object(4)
memory usage: 99.1+ KB


In [41]:
df

,종목코드,탭코드,종목명,현재가,등락구분,전일비,등락률,순자산가치(NAV),3개월수익률,거래량,거래대금(백만),시가총액(억),유형,브랜드,인버스,레버리지,환헤지H
0,459580,6,KODEX CD금리액티브(합성),1046085,2,230,0.02,1046089.0,0.8972,195638,204654,93402,채권,KODEX,False,False,False
1,069500,1,KODEX 200,36820,5,-115,-0.31,36885.0,-0.4931,1101467,40510,61140,국내 시장지수,KODEX,False,False,False
2,357870,6,TIGER CD금리투자KIS(합성),54825,3,0,0.00,54828.0,0.9110,25319,1388,61007,채권,TIGER,False,False,False
3,423160,6,KODEX KOFR금리액티브(합성),107820,2,30,0.03,107779.0,0.8939,16807,1812,45037,채권,KODEX,False,False,False
4,360750,4,TIGER 미국S&P500,18650,5,-35,-0.19,18599.0,3.6730,4820460,89909,43184,해외 주식,TIGER,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,267450,6,RISE 미국장기국채선물인버스(H),10160,3,0,0.00,10216.0,-5.6639,0,0,20,채권,RISE,True,False,True
877,412560,2,TIGER BBIG레버리지,2190,5,-10,-0.45,2199.0,0.9174,1411,3,18,국내 업종/테마,TIGER,False,True,False
878,333980,2,PLUS KS퀄리티가중TR,15230,2,20,0.13,15266.0,0.3960,4,0,15,국내 업종/테마,PLUS,False,False,False
879,436180,2,PLUS K-유니콘투자기업액티브,9975,2,25,0.25,9987.0,-3.5853,1,0,10,국내 업종/테마,PLUS,False,False,False


### 파일로 저장하기

In [42]:
# 기존에 불러왔던 파일명에서 _raw를 제거하고 새로운 파일명으로 저장합니다.
save_file_name = file_name.replace("_raw", "")
save_file_name

'data/etf_2024-08-23.csv'

In [43]:
# pandas의 to_csv로 파일을 저장합니다.
# index=False로 인덱스값은 저장하지 않습니다.
df.to_csv(save_file_name, index=False)

In [44]:
# 저장한 파일을 읽어와서 제대로 저장되었는지 확인합니다.
# 또, 종목코드는 그냥 읽어오면 숫자 맨 앞의 0이 생략될 수 있으니 object 타입으로 불러옵니다.
pd.read_csv(save_file_name, dtype={"종목코드": object})

,종목코드,탭코드,종목명,현재가,등락구분,전일비,등락률,순자산가치(NAV),3개월수익률,거래량,거래대금(백만),시가총액(억),유형,브랜드,인버스,레버리지,환헤지H
0,459580,6,KODEX CD금리액티브(합성),1046085,2,230,0.02,1046089.0,0.8972,195638,204654,93402,채권,KODEX,False,False,False
1,069500,1,KODEX 200,36820,5,-115,-0.31,36885.0,-0.4931,1101467,40510,61140,국내 시장지수,KODEX,False,False,False
2,357870,6,TIGER CD금리투자KIS(합성),54825,3,0,0.00,54828.0,0.9110,25319,1388,61007,채권,TIGER,False,False,False
3,423160,6,KODEX KOFR금리액티브(합성),107820,2,30,0.03,107779.0,0.8939,16807,1812,45037,채권,KODEX,False,False,False
4,360750,4,TIGER 미국S&P500,18650,5,-35,-0.19,18599.0,3.6730,4820460,89909,43184,해외 주식,TIGER,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,267450,6,RISE 미국장기국채선물인버스(H),10160,3,0,0.00,10216.0,-5.6639,0,0,20,채권,RISE,True,False,True
877,412560,2,TIGER BBIG레버리지,2190,5,-10,-0.45,2199.0,0.9174,1411,3,18,국내 업종/테마,TIGER,False,True,False
878,333980,2,PLUS KS퀄리티가중TR,15230,2,20,0.13,15266.0,0.3960,4,0,15,국내 업종/테마,PLUS,False,False,False
879,436180,2,PLUS K-유니콘투자기업액티브,9975,2,25,0.25,9987.0,-3.5853,1,0,10,국내 업종/테마,PLUS,False,False,False
